In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
#load the data
fifaData = '/Users/yonwu/Downloads/fifa-18-demo-player-dataset/CompleteDataset.csv'
df = pd.read_csv(fifaData)
df.head()

,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#check atrributes of the players data in the fifaData(check colomn index)
df.columns

Index(['Unnamed: 0', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking',
       'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM', 'ID',
       'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB',
       'Preferred Positions', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       'RS', 'RW', 'RWB', 'ST'],
      dtype='object')

In [19]:
#set a filter that only contains the attributes(foootablll skill related competence)
#that we need for a position prediction
#remove football unrealted attributes(name, age, photo, nationality, Flag...
atrributes_related = ['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking',
       'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys','Preferred Positions']

In [27]:
#User the filter to clean the noise of the data
df = df[atrributes_related]
df.head()

,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions
0,89,63,89,63,93,95,85,81,91,94,...,83,94,23,91,92,31,80,85,88,ST LW
1,92,48,90,95,95,96,77,89,97,95,...,88,85,26,87,73,28,59,90,85,RW
2,94,56,96,82,95,92,75,81,96,89,...,81,80,33,90,78,24,53,80,83,LW
3,88,78,86,60,91,83,77,86,86,94,...,83,87,38,77,89,45,80,84,88,ST
4,58,29,52,35,48,70,15,14,30,13,...,55,25,11,61,44,10,83,70,11,GK


In [33]:
#Continue the data pre processing and clean
#Now the data only contains the atrributes that we are intrested
#But we find 2 more steps we need to do, first is to handle multiple preferred positions
#the coresponding solution is to duplicated the data which each preferred position for one player
#Second is to perform caculations on some atrributes that contains "+ -"
df_add_duplicated_data = df.copy()
df_add_duplicated_data.drop(df_add_duplicated_data.index, inplace=True)
#Created a blank data file only contains the attributes
df_add_duplicated_data


,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions


In [34]:
#Duplicated data
unique_preperred_postions_list = df['Preferred Positions'].str.split().apply(lambda x: x[0]).unique()
#position_filter = False
for position in unique_preperred_postions_list:
    postion_filter = df['Preferred Positions'].str.contains(position)
    df_tmp = df[df['Preferred Positions'].str.contains(position)]
    df_tmp['Preferred Positions'] = position
    df_add_duplicated_data = df_add_duplicated_data.append(df_tmp)
df_add_duplicated_data.head()

/Users/yonwu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions
0,89,63,89,63,93,95,85,81,91,94,...,83,94,23,91,92,31,80,85,88,ST
3,88,78,86,60,91,83,77,86,86,94,...,83,87,38,77,89,45,80,84,88,ST
5,79,80,78,80,89,87,62,77,85,91,...,83,88,19,83,79,42,84,78,87,ST
9,78,50,75,69,85,86,68,74,84,91,...,75,88,18,80,72,22,85,70,88,ST
13,88,80,90,87,87,86,80,78,90,85,...,81,84,35,84,85,39,72,83,83,ST


In [35]:
#Now data is duplicated, next is to deal with "+ -"
attribute_list = df_add_duplicated_data.columns.values.tolist()
del attribute_list[-1]
attribute_list



['Acceleration',
 'Aggression',
 'Agility',
 'Balance',
 'Ball control',
 'Composure',
 'Crossing',
 'Curve',
 'Dribbling',
 'Finishing',
 'Free kick accuracy',
 'GK diving',
 'GK handling',
 'GK kicking',
 'GK positioning',
 'GK reflexes',
 'Heading accuracy',
 'Interceptions',
 'Jumping',
 'Long passing',
 'Long shots',
 'Marking',
 'Penalties',
 'Positioning',
 'Reactions',
 'Short passing',
 'Shot power',
 'Sliding tackle',
 'Sprint speed',
 'Stamina',
 'Standing tackle',
 'Strength',
 'Vision',
 'Volleys']

In [37]:
for attribute in attribute_list:
    df_add_duplicated_data[attribute] = df_add_duplicated_data[attribute].apply(lambda x: eval(x) if isinstance(x,str) else x)
df_add_duplicated_data.head()


,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions
0,89,63,89,63,93,95,85,81,91,94,...,83,94,23,91,92,31,80,85,88,ST
3,88,78,86,60,91,83,77,86,86,94,...,83,87,38,77,89,45,80,84,88,ST
5,79,80,78,80,89,87,62,77,85,91,...,83,88,19,83,79,42,84,78,87,ST
9,78,50,75,69,85,86,68,74,84,91,...,75,88,18,80,72,22,85,70,88,ST
13,88,80,90,87,87,86,80,78,90,85,...,81,84,35,84,85,39,72,83,83,ST


In [38]:
#After cleaning the data, next step is to do the data normalization
df_normalized = df_add_duplicated_data.iloc[:,:-1].div(df_add_duplicated_data.iloc[:,:-1].sum(axis=1), axis=0)
df_normalized['Preferred Positions'] = df_add_duplicated_data['Preferred Positions']
df_normalized.head()

,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions
0,0.038313,0.027120,0.038313,0.027120,0.040034,0.040895,0.036591,0.034869,0.039173,0.040465,...,0.035730,0.040465,0.009901,0.039173,0.039604,0.013345,0.034438,0.036591,0.037882,ST
3,0.037068,0.032856,0.036226,0.025274,0.038332,0.034962,0.032435,0.036226,0.036226,0.039596,...,0.034962,0.036647,0.016007,0.032435,0.037489,0.018955,0.033698,0.035383,0.037068,ST
5,0.035426,0.035874,0.034978,0.035874,0.039910,0.039013,0.027803,0.034529,0.038117,0.040807,...,0.037220,0.039462,0.008520,0.037220,0.035426,0.018834,0.037668,0.034978,0.039013,ST
9,0.038105,0.024426,0.036639,0.033708,0.041524,0.042013,0.033219,0.036150,0.041036,0.044455,...,0.036639,0.042990,0.008793,0.039082,0.035173,0.010747,0.041524,0.034196,0.042990,ST
13,0.038818,0.035289,0.039700,0.038377,0.038377,0.037936,0.035289,0.034407,0.039700,0.037494,...,0.035730,0.037053,0.015439,0.037053,0.037494,0.017203,0.031760,0.036612,0.036612,ST


In [62]:
X_train = df_normalized.iloc[:,:-1].to_numpy()
y_train = df_normalized.iloc[:,-1].to_numpy()
print('X train dataset shape: {}'.format(X_train.shape))
print('X test dataset shape: {}'.format(y_train.shape))

X train dataset shape: (29280, 34)
X test dataset shape: (29280,)


In [64]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

neural_network_classifier = MLPClassifier(random_state=0)
#neural_network_classifier = neural_network_position.fit(X_train, y_train)

for train, test in KFold(n_splits=5, shuffle=True).split(X_train):
    X_po_train = X_train[train]
    X_po_test = X_train[test]
    y_po_train = y_train[train]
    y_po_test = y_train[test]
    
    neural_network_classifier.fit(X_po_train, y_po_train)
    print('X train dataset shape: {}'.format(X_po_train.shape))
    print('X test dataset shape: {}'.format(X_po_test.shape))
    print('y train dataset shape: {}'.format(y_po_train.shape))
    print('y test dataset shape: {}'.format(y_po_test.shape))
  
    print("Accuracy: %.1f%%" %(100*neural_network_classifier.score(X_po_test, y_po_test)))
    



/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


X train dataset shape: (23424, 34)
X test dataset shape: (5856, 34)
y train dataset shape: (23424,)
y test dataset shape: (5856,)
Accuracy: 48.0%


/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


X train dataset shape: (23424, 34)
X test dataset shape: (5856, 34)
y train dataset shape: (23424,)
y test dataset shape: (5856,)
Accuracy: 48.2%


/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


X train dataset shape: (23424, 34)
X test dataset shape: (5856, 34)
y train dataset shape: (23424,)
y test dataset shape: (5856,)
Accuracy: 49.6%


/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


X train dataset shape: (23424, 34)
X test dataset shape: (5856, 34)
y train dataset shape: (23424,)
y test dataset shape: (5856,)
Accuracy: 48.2%
X train dataset shape: (23424, 34)
X test dataset shape: (5856, 34)
y train dataset shape: (23424,)
y test dataset shape: (5856,)
Accuracy: 48.4%


/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train_po, X_test_po, y_train_po, y_test_po = train_test_split(X_train, y_train, train_size=.8)
neural_network_classifier.fit(X_po_train, y_po_train)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

c_matrix_report = classification_report(y_test_po, neural_network_classifier.predict(X_test_po), output_dict=True)
import pandas
df = pandas.DataFrame(c_matrix_report).transpose()
df

/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/yonwu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
CAM,0.336493,0.331002,0.333725,429.000000
CB,0.684148,0.841642,0.754767,682.000000
CDM,0.466667,0.409434,0.436181,530.000000
CF,0.000000,0.000000,0.000000,62.000000
CM,0.437773,0.549315,0.487242,730.000000
GK,1.000000,1.000000,1.000000,416.000000
LB,0.336134,0.568720,0.422535,422.000000
LM,0.273556,0.344828,0.305085,522.000000
LW,0.000000,0.000000,0.000000,190.000000
LWB,0.000000,0.000000,0.000000,31.000000


In [69]:
w = neural_network_classifier.coefs_

In [72]:
w

[array([[-5.33080950e-001, -5.05832438e-001,  3.06312061e-198, ...,
          1.78236088e-315,  4.30170710e-316, -1.41945781e+000],
        [ 6.33623717e-002, -4.28738215e-001, -9.77096952e-192, ...,
         -9.77319767e-316,  8.13822595e-316,  8.07541767e-001],
        [ 6.42364538e-002,  2.73150899e-001,  1.65475088e-199, ...,
         -7.73927757e-316, -1.11217153e-315,  2.36333138e-001],
        ...,
        [ 3.16208572e-001, -2.67313273e-001, -8.11007387e-190, ...,
          2.88292141e-316, -1.77213026e-315,  5.32431891e-001],
        [ 2.14442423e+000,  2.25441433e+000, -8.24716425e-202, ...,
          2.46859268e-315, -1.61957887e-315,  2.36144652e+000],
        [-1.28354932e+000, -1.39891646e+000, -1.46309111e-211, ...,
         -1.52917919e-315, -1.03594374e-315,  1.06228054e+000]]),
 array([[ 1.28201797e+000,  6.22093015e-001,  1.64179165e+000, ...,
         -1.41126143e+000, -6.59621309e-001, -2.24143792e+000],
        [ 8.10926632e-001, -1.51200723e-001,  1.08640967e+000